## Imports

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from sklearn.metrics import confusion_matrix

## Set directories

In [ ]:
DIR_UNIL = "" # File path to folder containing UNIL images
DIR_CROP = "" # File path to folder containing CROP images

# File path for Excel file containing UNIL train/val/test splitting:
UNIL_TRAIN_VALID_TEST_EXCEL_FILEPATH = "............/classification_training_validation_test_UNIL.xlsx"
# File path for Excel file containing CROP train/val/test splitting:
CROP_TRAIN_VALID_TEST_EXCEL_FILEPATH = "............/classification_training_validation_test_CROP.xlsx"

# File path for neural net 1 (neural net taking UNIL inputs) for ensembling
DIR_NN1 = "............./0001_InceptionV3_imagenet_FINALEPOCH_UNIL.h5"
# File path for neural net 2 (neural net taking CROP inputs) for ensembling
DIR_NN2 = "............./0001_Xception_imagenet_FINALEPOCH_CROP.h5"


In [ ]:
test_df_unil = pd.read_excel(UNIL_TRAIN_VALID_TEST_EXCEL_FILEPATH, 
                             sheet_name=f"TEST_DF", dtype=str)
test_df_crop = pd.read_excel(CROP_TRAIN_VALID_TEST_EXCEL_FILEPATH, 
                             sheet_name=f"TEST_DF", dtype=str)

image_count = test_df_unil.shape[0]

## Setup image data generators and load models

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator_1 = test_datagen.flow_from_dataframe(test_df_unil, directory=DIR_UNIL, class_mode='categorical', x_col='filenames', y_col='labels',
                                                    target_size=(299, 299), batch_size=1, shuffle=False)
test_generator_2 = test_datagen.flow_from_dataframe(test_df_crop, directory=DIR_CROP, class_mode='categorical', x_col='filenames', y_col='labels',
                                                    target_size=(299, 299), batch_size=1, shuffle=False)

In [ ]:
model_1 = load_model(DIR_NN1)
model_2 = load_model(DIR_NN2)

## Generate class predictions and true classes

In [ ]:
# Generate numpy array of predictions (one hot encoded) for model 1 as well as array of true class labels
output_model_1 = []
output_true = []

for i, (image, true) in enumerate(tqdm(test_generator_1)):
    pred = model_1.predict(image)[0]
    output_model_1.append(pred)
    output_true.append(true)
    if i == image_count-1:
        break

# Generate numpy array of predictions (one hot encoded) for model 2       
output_model_2 = []

for i, (image, _) in enumerate(tqdm(test_generator_2)):
    pred = model_2.predict(image)[0]
    output_model_2.append(pred)
    if i == image_count-1:
        break

# Generate 1:1 weighted averages of predictions for the two models
weighted_averages = []
for (out1, out2) in zip(output_model_1, output_model_2):
    weighted_average = (out1+out2)/2
    weighted_averages.append(weighted_average)

## Calculate accuracy and top 3 accuracy

In [ ]:
corrects = []
for i in range(image_count):
    corrects.append(np.argmax(weighted_averages[i]) == np.argmax(output_true[i]))

print(f"Accuracy: {sum(corrects)/image_count}")

In [ ]:
top_3_accuracy = []
pred_list = []
true_list = []
for i in range(image_count):
    pred = weighted_averages[i]
    true = output_true[i]
    top_3_pred = np.argpartition(pred, -3)[-3:]
    if top_3_pred[0] == np.argmax(true) or top_3_pred[1] == np.argmax(true) or top_3_pred[2] == np.argmax(true):
        correct = True
    else:
        correct = False
    top_3_accuracy.append(correct)
    pred_list.append(np.argmax(pred))
    true_list.append(np.argmax(true))
    

top_3_accuracy = sum(top_3_accuracy) / image_count
print(f"Top 3 accuracy: {top_3_accuracy}")

## Generate confusion matrix

In [ ]:
# Generate confusion matrix numpy array
confusion_mx = confusion_matrix(true_list,pred_list)

class_indices = test_generator_1.class_indices

# Then convert confusion matrix into pd dataframe with labels along column and along top
class_indices = test_generator_1.class_indices
confusion_matrix_df = pd.DataFrame(confusion_mx,
                                   index = [f"{i}_{c}_true" for i,c in enumerate(list(class_indices.keys()))],
                                   columns = [f"{i}" for i,c in enumerate(list(class_indices.keys()))])

print(confusion_matrix_df)